In [ ]:
from raphtory import Graph
from raphtory import algorithms
from raphtory import graph_loader
import time
import sys

g = graph_loader.stable_coin_graph("/tmp/stablecoin", 1)

max_size = sys.maxsize
min_size = -sys.maxsize - 1

start_time = time.time()
res = algorithms.pagerank(g, min_size, max_size, 20)
end_time = time.time()

print("Time taken in secs", end_time - start_time)
# print(res)